In [1]:
import torch.nn as nn
import torch.nn.functional as F
import torch
import torch.optim as optim
from torchvision.datasets import FashionMNIST
from torchvision import transforms
from torchvision import datasets, transforms

import numpy as np

In [52]:
train_dataset = FashionMNIST(root=".", train=True, download=True, transform=transforms.ToTensor())
test_dataset = FashionMNIST(root=".", train=False, download=True, transform=transforms.ToTensor())

In [56]:
train_dataset[0][0].shape

torch.Size([1, 28, 28])

In [177]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.class_size = 10
        self.conv_layer_1 =  nn.Conv2d(1, 14, kernel_size=11, stride=2, padding=1, dilation=1)
        #self.pool = nn.MaxPool2d(2, 2)
        # self.conv_layer_2 = nn.Conv2d(16, 10, (3, 3), 3, 3)
        self.conv_layer_2 = nn.Conv2d(14, 10, kernel_size=11, stride=2, padding=1, dilation=1)
        self.conv_layer_3 = nn.Conv2d(10, 10, kernel_size=1)
        
    def forward(self, input_image):
        result_1 = F.relu(self.conv_layer_1(input_image))
        #print(f'{result_1.shape}: {result_1}')
        result_2 = F.relu(self.conv_layer_2(result_1))
        result_3 = F.relu(self.conv_layer_3(result_2))
        #return F.softmax(result_3, dim=1).squeeze(2).squeeze(2)
        return F.log_softmax(result_3, dim=1).squeeze(2).squeeze(2)
    
class ModelMask(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.class_size = 10
        self.conv_layer_1 =  nn.Conv2d(1, 14, kernel_size=11, stride=2, padding=1, dilation=1)
        #self.pool = nn.MaxPool2d(2, 2)
        # self.conv_layer_2 = nn.Conv2d(16, 10, (3, 3), 3, 3)
        self.conv_layer_2 = nn.Conv2d(14, 10, kernel_size=11, stride=2, padding=1, dilation=1)
        self.conv_layer_3 = nn.Conv2d(10, 10, kernel_size=1)
        
    def forward(self, input_image):
        result_1 = F.relu(self.conv_layer_1(input_image))
        #print(f'{result_1.shape}: {result_1}')
        result_2 = F.relu(self.conv_layer_2(result_1))
        result_3 = F.relu(self.conv_layer_3(result_2))
        #return F.softmax(result_3, dim=1).squeeze(2).squeeze(2)
        return F.log_softmax(result_3, dim=1).squeeze(2).squeeze(2)
    

In [178]:
device = 0

model = Model().to(device)

net = Net().to(device)

In [179]:

# model.forward(image.view(28, 28, 1, 1))
#a = model.forward(image.view(-1))
a = model(train_dataset[0][0].view(1, 1, 28, 28).to(device))
#pixels = np.asarray(train_dataset[0][0])


In [180]:
a.shape

torch.Size([1, 10])

In [181]:
a.squeeze(2).squeeze(2).shape

RuntimeError: Dimension out of range (expected to be in range of [-2, 1], but got 2)

In [185]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        # print(f'CICCIO {output.shape}, {target.shape}')
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 50 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

def main():
    use_cuda = 0
    device = torch.device("cuda" if use_cuda else "cpu")

    kwargs = {'num_workers': 2, 'pin_memory': True} if use_cuda else {}
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
        ])),
        batch_size=100, shuffle=True, **kwargs)
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
])),
        batch_size=10, shuffle=True, **kwargs)


    model = Model().to(device)
    optimizer = optim.SGD(model.parameters(), lr=0.003, momentum=0.9, nesterov=True)

    for epoch in range(1, 10 + 1):
        train(model, device, train_loader, optimizer, epoch)
        test(model, device, test_loader)
main()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.304036
Train Epoch: 1 [5000/60000 (8%)]	Loss: 0.712879
Train Epoch: 1 [10000/60000 (17%)]	Loss: 0.583876
Train Epoch: 1 [15000/60000 (25%)]	Loss: 0.416198
Train Epoch: 1 [20000/60000 (33%)]	Loss: 0.305618
Train Epoch: 1 [25000/60000 (42%)]	Loss: 0.458283
Train Epoch: 1 [30000/60000 (50%)]	Loss: 0.550277
Train Epoch: 1 [35000/60000 (58%)]	Loss: 0.375713
Train Epoch: 1 [40000/60000 (67%)]	Loss: 0.436687
Train Epoch: 1 [45000/60000 (75%)]	Loss: 0.602333
Train Epoch: 1 [50000/60000 (83%)]	Loss: 0.282566
Train Epoch: 1 [55000/60000 (92%)]	Loss: 0.238015

Test set: Average loss: 0.3306, Accuracy: 8740/10000 (87%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.354596
Train Epoch: 2 [5000/60000 (8%)]	Loss: 0.337276
Train Epoch: 2 [10000/60000 (17%)]	Loss: 0.188028
Train Epoch: 2 [15000/60000 (25%)]	Loss: 0.423722
Train Epoch: 2 [20000/60000 (33%)]	Loss: 0.325596
Train Epoch: 2 [25000/60000 (42%)]	Loss: 0.307845
Train Epoch: 2 [30000/60000 (50%)]	Loss: 0.235111
Tra

In [ ]:
    l = datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))]))

In [63]:
l

Dataset MNIST
    Number of datapoints: 60000
    Split: train
    Root Location: ../data
    Transforms (if any): Compose(
                             ToTensor()
                             Normalize(mean=(0.1307,), std=(0.3081,))
                         )
    Target Transforms (if any): None

In [66]:
l[0][0].shape

torch.Size([1, 28, 28])

In [6]:
cap = datasets.CocoDetection(root = '/home/none/documents/external/datasets/coco/train2017',
                        annFile = '/home/none/documents/external/datasets/annotations/stuff_train2017.json',
                        transform=transforms.ToTensor())

loading annotations into memory...
Done (t=8.64s)
creating index...
index created!


In [ ]:
coco_dataset = []
for i in range(len(cap)):
    s = None
    try:
        s = cap[i]
    except BaseException as e:
        print(f'CC is {i} {e}')
        ()
    if s is not None:
        coco_dataset.append(s)
        #break
        

CC is 0 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000391895.jpg'
CC is 1 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000522418.jpg'
CC is 2 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000184613.jpg'
CC is 3 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000318219.jpg'
CC is 4 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000554625.jpg'
CC is 5 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000574769.jpg'
CC is 7 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000309022.jpg'
CC is 9 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000222564.jpg'
CC is 10 [Errno 2] No such file or directory: '/home/non

CC is 142 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000281221.jpg'
CC is 143 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000108169.jpg'
CC is 144 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000440575.jpg'
CC is 145 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000354533.jpg'
CC is 146 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000545959.jpg'
CC is 147 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000511058.jpg'
CC is 148 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000091227.jpg'
CC is 149 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000242139.jpg'
CC is 150 [Errno 2] No such file or dire

CC is 220 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000125997.jpg'
CC is 221 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000069911.jpg'
CC is 222 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000573291.jpg'
CC is 223 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000322226.jpg'
CC is 224 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000161470.jpg'
CC is 225 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000452224.jpg'
CC is 226 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000322654.jpg'
CC is 227 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000110196.jpg'
CC is 228 [Errno 2] No such file or dire

CC is 309 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000209692.jpg'
CC is 310 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000205378.jpg'
CC is 311 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000551952.jpg'
CC is 312 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000535292.jpg'
CC is 314 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000143824.jpg'
CC is 316 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000168706.jpg'
CC is 317 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000414679.jpg'
CC is 318 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000486203.jpg'
CC is 320 [Errno 2] No such file or dire

CC is 467 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000256091.jpg'
CC is 468 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000433460.jpg'
CC is 469 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000307569.jpg'
CC is 470 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000181962.jpg'
CC is 471 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000120935.jpg'
CC is 472 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000405778.jpg'
CC is 473 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000520727.jpg'
CC is 475 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000353807.jpg'
CC is 476 [Errno 2] No such file or dire

...

CC is 2587 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000437497.jpg'
CC is 2588 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000412658.jpg'
CC is 2589 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000384401.jpg'
CC is 2590 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000182503.jpg'
CC is 2591 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000499577.jpg'
CC is 2592 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000187352.jpg'
CC is 2593 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000461883.jpg'
CC is 2594 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000206512.jpg'
CC is 2595 [Errno 2] No such fil

CC is 2743 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000507249.jpg'
CC is 2744 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000075923.jpg'
CC is 2745 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000240918.jpg'
CC is 2746 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000184700.jpg'
CC is 2747 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000191691.jpg'
CC is 2748 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000130732.jpg'
CC is 2749 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000304389.jpg'
CC is 2750 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000196793.jpg'
CC is 2751 [Errno 2] No such fil

CC is 2856 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000463132.jpg'
CC is 2857 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000218224.jpg'
CC is 2858 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000459502.jpg'
CC is 2859 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000078580.jpg'
CC is 2860 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000368611.jpg'
CC is 2861 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000345329.jpg'
CC is 2862 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000118728.jpg'
CC is 2863 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000091667.jpg'
CC is 2864 [Errno 2] No such fil

CC is 3010 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000495014.jpg'
CC is 3011 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000167510.jpg'
CC is 3012 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000328785.jpg'
CC is 3014 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000564739.jpg'
CC is 3015 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000101096.jpg'
CC is 3016 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000073731.jpg'
CC is 3017 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000460813.jpg'
CC is 3018 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000351904.jpg'
CC is 3019 [Errno 2] No such fil

CC is 3103 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000151358.jpg'
CC is 3104 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000153288.jpg'
CC is 3106 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000076721.jpg'
CC is 3107 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000319768.jpg'
CC is 3108 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000412813.jpg'
CC is 3109 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000532867.jpg'
CC is 3110 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000550051.jpg'
CC is 3111 [Errno 2] No such file or directory: '/home/none/documents/external/datasets/coco/train2017/000000462953.jpg'
CC is 3112 [Errno 2] No such fil

In [4]:
s[1][0]['segmentation']['counts']

'c9n4Q:O10000000000O0101O0000O1001OO1010N100001N1O1WKnEd4S:[KoEd4X:M10O0100O1O010O100O00100O10O0100O1O010O100O10O10K^ElKc:g3^E`L1Ha:h3_E_L1H`:h3`E^L2J]:h3dE[Ld:d3]E[Le:d3\\EZLc:g3=M00100O001O10O0100O2N10kDbLi:_3UEcLS:M\\F_3@dLS:0\\F]3_OeLS:O_F^3ZOdLV:O`Fd3^9`LbF_3]9bLbF^3`9aLaF^3b9`L\\Fa3e9^L[Fb3e9^L[Fb3e9_LZF`3g9`LYF_3h9aLYF_3f9aLZF`3e9`LZFX3_OaLS:0RF8<W3e9aLoE8;W3g9aLnE99U3j9cLmE97Q3Q:eLhE97S3Z:PMbE[2HhMNAi:d0XES22hMMAi:^3ZEcL08MGa:0aE2N[3;cLE4e:a3jE[LX:e3`014K4M1O0OGdLQE\\3o:dLQE\\3o:dLQE[3P;eLPE[3Z;0000O100F_DVMa;j2`DTMa;k2bDSM^;g2`0bMUDh1k;XNWDb1m;]NUD_1n;aNRD\\1P<eNPDZ1Q<eNQDZ1o;fNQDZ1o;fNRDX1k;lNVDS1f;POcD_OXOU1S<^OkD`0T;AmD<U;DmD3mNYO[<d0iD2Y;MjD1V;NkD2V;MkD2V;MjD@gNa0`<NkD2U;NkD2U;NjD3V;MkD2U;NkD2U;NkD2V;J`CE\\17`;JRUh40^lWKc0J6kA0P=`1_O2M5L3N2M3M3N2M3L4GkLbDZ3_;3000000O2O0000000O2M101O2M2O1O0O13KO21N1^OiD^MW;g1gETNZ:j1kESNZ:IeD0Of1R1_Nf:a1[E\\Ng:c1[1EiBiN\\=W17N10000O1O1\\Od0O1N3L3M3K5N2000Vjg1'

In [2]:
from pycocotools.coco import COCO

In [5]:
coco = COCO('/home/none/documents/external/datasets/annotations/stuff_train2017.json')

loading annotations into memory...
Done (t=21.47s)
creating index...
index created!


In [15]:
mask = coco.annToMask(s[1][0])
image = s[0]

In [9]:
s[1][0]

{'segmentation': {'counts': 'c9n4Q:O10000000000O0101O0000O1001OO1010N100001N1O1WKnEd4S:[KoEd4X:M10O0100O1O010O100O00100O10O0100O1O010O100O10O10K^ElKc:g3^E`L1Ha:h3_E_L1H`:h3`E^L2J]:h3dE[Ld:d3]E[Le:d3\\EZLc:g3=M00100O001O10O0100O2N10kDbLi:_3UEcLS:M\\F_3@dLS:0\\F]3_OeLS:O_F^3ZOdLV:O`Fd3^9`LbF_3]9bLbF^3`9aLaF^3b9`L\\Fa3e9^L[Fb3e9^L[Fb3e9_LZF`3g9`LYF_3h9aLYF_3f9aLZF`3e9`LZFX3_OaLS:0RF8<W3e9aLoE8;W3g9aLnE99U3j9cLmE97Q3Q:eLhE97S3Z:PMbE[2HhMNAi:d0XES22hMMAi:^3ZEcL08MGa:0aE2N[3;cLE4e:a3jE[LX:e3`014K4M1O0OGdLQE\\3o:dLQE\\3o:dLQE[3P;eLPE[3Z;0000O100F_DVMa;j2`DTMa;k2bDSM^;g2`0bMUDh1k;XNWDb1m;]NUD_1n;aNRD\\1P<eNPDZ1Q<eNQDZ1o;fNQDZ1o;fNRDX1k;lNVDS1f;POcD_OXOU1S<^OkD`0T;AmD<U;DmD3mNYO[<d0iD2Y;MjD1V;NkD2V;MkD2V;MjD@gNa0`<NkD2U;NkD2U;NjD3V;MkD2U;NkD2U;NkD2V;J`CE\\17`;JRUh40^lWKc0J6kA0P=`1_O2M5L3N2M3M3N2M3L4GkLbDZ3_;3000000O2O0000000O2M101O2M2O1O0O13KO21N1^OiD^MW;g1gETNZ:j1kESNZ:IeD0Of1R1_Nf:a1[E\\Ng:c1[1EiBiN\\=W17N10000O1O1\\Od0O1N3L3M3K5N2000Vjg1',
  'size': [479, 640]},
 'area': 20396.0,
 'iscrowd':

In [10]:
mask.sum()

20396

In [14]:
s[0]

tensor([[[0.5490, 0.5451, 0.5529,  ..., 0.4863, 0.4824, 0.4667],
         [0.5373, 0.5451, 0.5529,  ..., 0.4784, 0.4941, 0.4902],
         [0.5333, 0.5451, 0.5490,  ..., 0.4784, 0.4863, 0.4824],
         ...,
         [0.4745, 0.4627, 0.4706,  ..., 0.5137, 0.5412, 0.5255],
         [0.5255, 0.5098, 0.4902,  ..., 0.5176, 0.5176, 0.4941],
         [0.4784, 0.4549, 0.5176,  ..., 0.4980, 0.5137, 0.5137]],

        [[0.4980, 0.4980, 0.5059,  ..., 0.4588, 0.4549, 0.4392],
         [0.4902, 0.4980, 0.5059,  ..., 0.4510, 0.4667, 0.4627],
         [0.4863, 0.4980, 0.5020,  ..., 0.4510, 0.4588, 0.4549],
         ...,
         [0.4471, 0.4353, 0.4353,  ..., 0.4941, 0.5137, 0.5020],
         [0.5059, 0.4824, 0.4549,  ..., 0.4980, 0.5020, 0.4667],
         [0.4510, 0.4275, 0.4824,  ..., 0.4784, 0.4941, 0.4980]],

        [[0.4235, 0.4118, 0.4118,  ..., 0.3882, 0.3843, 0.3686],
         [0.4039, 0.4118, 0.4118,  ..., 0.3882, 0.3961, 0.3922],
         [0.3922, 0.4039, 0.4078,  ..., 0.3882, 0.3961, 0.